In [ ]:
import control_experiment as experiment_runner
from pathlib import Path
import numpy as np
import proplot as pplt
import xarray as xr
import pandas as pd
from utils.files import OIFSPreprocessor, NEMOPreprocessor

In [ ]:
oifs_preprocessor = OIFSPreprocessor(
    experiment_runner.start_date, np.timedelta64(-7, "h")
)
nemo_preprocessor = NEMOPreprocessor(
    experiment_runner.start_date, np.timedelta64(-7, "h")
)

In [ ]:
plotting_output_directory = Path("plots/control_experiment_1")
plotting_output_directory.mkdir(exist_ok=True)

In [ ]:
naive_exp_ids = ["C1N0", "C1N1", "C1N2"]
schwarz_exp_id = "C1S0"
max_schwarz_iters = experiment_runner.max_iters
setup = "PAPA"

In [ ]:
oifs_progvars_naive = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/progvar.nc", preprocess=oifs_preprocessor.preprocess
    )
    for exp_id in naive_exp_ids
]
oifs_diagvars_naive = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/diagvar.nc", preprocess=oifs_preprocessor.preprocess
    )
    for exp_id in naive_exp_ids
]
nemo_t_grids_naive = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/{exp_id}*_T.nc", preprocess=nemo_preprocessor.preprocess
    )
    for exp_id in naive_exp_ids
]
nemo_u_grids_naive = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/{exp_id}*_U.nc", preprocess=nemo_preprocessor.preprocess
    )
    for exp_id in naive_exp_ids
]
nemo_v_grids_naive = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/{exp_id}*_V.nc", preprocess=nemo_preprocessor.preprocess
    )
    for exp_id in naive_exp_ids
]

oifs_progvar_cvg_swz = xr.open_mfdataset(
    f"{setup}/{schwarz_exp_id}_{max_schwarz_iters}/progvar.nc",
    preprocess=oifs_preprocessor.preprocess,
)
oifs_diagvar_cvg_swz = xr.open_mfdataset(
    f"{setup}/{schwarz_exp_id}_{max_schwarz_iters}/diagvar.nc",
    preprocess=oifs_preprocessor.preprocess,
)
nemo_t_grid_cvg_swz = xr.open_mfdataset(
    f"{setup}/{schwarz_exp_id}_{max_schwarz_iters}/{schwarz_exp_id}*_T.nc",
    preprocess=nemo_preprocessor.preprocess,
)
nemo_u_grid_cvg_swz = xr.open_mfdataset(
    f"{setup}/{schwarz_exp_id}_{max_schwarz_iters}/{schwarz_exp_id}*_U.nc",
    preprocess=nemo_preprocessor.preprocess,
)
nemo_v_grid_cvg_swz = xr.open_mfdataset(
    f"{setup}/{schwarz_exp_id}_{max_schwarz_iters}/{schwarz_exp_id}*_V.nc",
    preprocess=nemo_preprocessor.preprocess,
)

oifs_diagvars_add_swz = []
for iter in range(1, max_schwarz_iters + 1):
    oifs_diagvars_add_swz.append(
        xr.open_mfdataset(
            f"{setup}/{schwarz_exp_id}_{iter}/diagvar.nc",
            preprocess=oifs_preprocessor.preprocess,
        )
    )

# Boundary Layer Type

## Boundary Layer Type for Thesis

In [ ]:
oifs_diagvars = [*oifs_diagvars_naive, oifs_diagvar_cvg_swz]

fig, axs = pplt.subplots(nrows=2, width="70em", height="37em", spany=False)
axs.format(abc="a)")

ax = axs[0]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atmosphere-first", "ocean-first", "converged SWR"]
shifts = [-0.12, -0.04, 0.04, 0.12]
for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    shift = shifts[i]
    ax.scatter(
        oifs_diagvar.pbl_type + shift,
        color=color,
        label=label,
        marker=".",
    )
ax.format(
    title="Boundary Layer Type in Control Experiment",
    ylabel="Boundary Layer Type",
    yticks=[0, 1, 2, 3],
    yticklabels=["DS", "DC", "Sc", "Cu"],
    ytickminor=False,
    xrotation=30,
    xlabel="Time",
)
ax.legend(ncol=1)

ax = axs[1]

pbl_type_changes = oifs_diagvars_add_swz[0].pbl_type.copy()
pbl_type_changes.data[:] = 0
pbl_types = [oifs_diagvar.pbl_type for oifs_diagvar in oifs_diagvars_add_swz]

previous_pbl_type = pbl_types[0]
for iter in range(1, max_schwarz_iters):
    pbl_type = pbl_types[iter]
    pbl_type_not_equal = np.where(pbl_type.data != previous_pbl_type.data)
    pbl_type_changes.data[pbl_type_not_equal] += 1
    previous_pbl_type = pbl_type
ax.scatter(
    pbl_type_changes,
    color="k",
    marker=".",
)
ax.format(
    ylabel="# of Boundary Layer Type Changes",
    ytickminor=False,
    title="Boundary Layer Type Changes during Schwarz Iterations",
    xlabel="Time",
    xrotation=30,
)
# ax.set_title("+1 if boundary layer type changed between subsequent iterations")

fig.savefig(plotting_output_directory / "ce_pbl_types.pdf")

## Pure BL Type Plot

## BL Type over Iteration

# Boundary Layer Heights

## Atmosphere

In [ ]:
oifs_diagvars = [*oifs_diagvars_naive, oifs_diagvar_cvg_swz]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, ax = pplt.subplots(width="70em", height="40em")

for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.pbl_height,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(
    ylabel=r"Boundary Layer Height $[m]$",
    xlabel="Time",
    title="Boundary Layer Height (Control Experiment)",
)
ax.legend(ncols=1)
fig.savefig(plotting_output_directory / "pbl_height.pdf")

In [ ]:
fig, ax = pplt.subplots(width="70em", height="40em")

oifs_diagvar = oifs_diagvar_cvg_swz
ax.plot(oifs_diagvar.height_f.sel(nlev=51), color="k")
ax.format(
    title="Height at Atmospheric Model Level 51", xlabel="Time", ylabel="Height [m]"
)
fig.savefig(plotting_output_directory / "height_L51.pdf")

In [ ]:
pressure_l51 = oifs_diagvar.pressure_f.sel(nlev=51)[0].load()
print(f"Pressure on Level 51: {pressure_l51.data[()]}")

$\Rightarrow$ for a vertical profile of the atmospheric boundary layer, the last 10 atmospheric levels should suffice!

## Ocean

According to [this report, p. 18](https://www.drakkar-ocean.eu/publications/reports/orca025-grd100-report-dussin), the first 10 levels should suffice for NEMO as well.

In [ ]:
nemo_t_grids = [*nemo_t_grids_naive, nemo_t_grid_cvg_swz]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, ax = pplt.subplots(width="70em", height="40em")

for i in range(len(colors)):
    nemo_t_grid = nemo_t_grids[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        nemo_t_grid.somxl010,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )

ax.format(
    ylim=[0, 10],
    ylabel="Mixed Layer Depth [m]",
    xlabel="Time",
    title="Mixed Layer Depth (Control Experiment)",
)

ax.legend(ncols=1)
fig.savefig(plotting_output_directory / "mldepth.pdf")

# Surface Heat Fluxes

In [ ]:
oifs_diagvars = [*oifs_diagvars_naive, oifs_diagvar_cvg_swz]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atmosphere-first", "ocean-first", "converged SWR"]
linestyles = ["--", ":", "-.", "-"]

fig, axs = pplt.subplots(
    nrows=2, ncols=2, height="45em", width="65em", sharey=2, spany=False
)

ims = []
ax = axs[0, 0]
for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    im = ax.plot(
        oifs_diagvar.sfc_sen_flx,
        color=color,
        label=label,
        ls=linestyle,
    )
    ims.append(im)
ax.format(
    ylabel=r"Heat Flux $[W \; m^{-2}]$",
    title="Surface Sensible Heat Flux",
    xlabel="Time",
)

ax = axs[0, 1]
for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.sfc_lat_flx,
        color=color,
        label=label,
        ls=linestyle,
    )
ax.format(
    ylabel=r"Heat Flux $[W \; m^{-2}]$", title="Surface Latent Heat Flux", xlabel="Time"
)

ax = axs[1, 0]
for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.sfc_swrad,
        color=color,
        label=label,
        ls=linestyle,
    )
ax.format(
    ylabel=r"Radiation $[W \; m^{-2}]$",
    title="Net Surface Shortwave Radiation",
    xlabel="Time",
)

ax = axs[1, 1]
for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.sfc_lwrad,
        color=color,
        label=label,
        ls=linestyle,
    )
ax.format(
    ylabel=r"Radiation $[W \; m^{-2}]$",
    title="Net Surface Longwave Radiation",
    xlabel="Time",
)

axs.format(abc="a)", xrotation=30)
fig.legend(ims, frame=False, ncols=4, loc="b")

fig.savefig(plotting_output_directory / "ce_surface_heat_fluxes.pdf")

# Cloud Cover

In [ ]:
oifs_diagvars = [*oifs_diagvars_naive, oifs_diagvar_cvg_swz]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, ax = pplt.subplots(width="70em", height="40em")

for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.total_cloud,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(
    title="Total Cloud Cover (Control Experiment)", xlabel="Time", ylabel="Cloud Cover"
)
ax.legend(ncols=1)
fig.savefig(plotting_output_directory / "ccover.pdf")

# Vertical Profiles

## Atmosphere

### Vertical Temperature Profile

In [ ]:
oifs_progvars = [*oifs_progvars_naive, oifs_progvar_cvg_swz]

titles = [
    "Parallel",
    "Sequential Atmosphere-First",
    "Sequential Ocean-First",
    "Converged Schwarz Waveform Relaxation",
]

fig, axs = pplt.subplots(nrows=4, ncols=1, height="90em", width="70em")

for i in range(len(oifs_progvars)):
    ax = axs[i]
    ax_title = titles[i]
    oifs_progvar = oifs_progvars[i]
    oifs_progvar = oifs_progvar.assign_coords(
        air_pressure=("nlev", oifs_progvar.pressure_f[0].data / 100)
    )
    oifs_progvar = oifs_progvar.swap_dims({"nlev": "air_pressure"})
    im = ax.contourf(
        oifs_progvar.t[:, 50:] - 273.15,
        levels=14,
        vmin=0,
        vmax=14,
        transpose=True,
        discrete=True,
    )
    ax.format(xlabel="Time", ylabel="Air Pressure [hPa]", title=ax_title)
    ax.invert_yaxis()
axs.format(
    suptitle="Vertical Atmospheric Temperature Profiles in BL (Control Experiment)"
)
fig.colorbar(im, loc="b", title="Temperature [°C]")
fig.savefig(plotting_output_directory / "air_temperature_profile.pdf")

### Vertical Humidity Profile

In [ ]:
oifs_progvars = [*oifs_progvars_naive, oifs_progvar_cvg_swz]

titles = [
    "Parallel",
    "Sequential Atmosphere-First",
    "Sequential Ocean-First",
    "Converged Schwarz Waveform Relaxation",
]

fig, axs = pplt.subplots(nrows=4, ncols=1, width="70em", height="90em")
axs.format(suptitle="Vertical Atmospheric Humidity Profiles in BL (Control Experiment)")

for i in range(len(oifs_progvars)):
    ax = axs[i]
    ax_title = titles[i]
    oifs_progvar = oifs_progvars[i]
    oifs_progvar = oifs_progvar.assign_coords(
        air_pressure=("nlev", oifs_progvar.pressure_f[0].data / 100)
    )
    oifs_progvar = oifs_progvar.swap_dims({"nlev": "air_pressure"})
    im = ax.contourf(
        oifs_progvar.q[:, 50:] * 1e3,
        levels=12,
        transpose=True,
        discrete=True,
    )
    ax.set_ylabel("Air Pressure [hPa]")
    ax.invert_yaxis()
    ax.set_title(ax_title)
fig.colorbar(im, loc="b", title="Water Vapor Mixing Ratio [g/kg]")
fig.savefig(plotting_output_directory / "q_profile.pdf")

## Ocean

### Vertical Temperature Profile

In [ ]:
nemo_t_grids = [*nemo_t_grids_naive, nemo_t_grid_cvg_swz]

titles = [
    "Parallel",
    "Sequential Atmosphere-First",
    "Sequential Ocean-First",
    "Converged Schwarz Waveform Relaxation",
]

fig, axs = pplt.subplots(nrows=4, ncols=1, width="70em", height="90em")
axs.format(
    suptitle="Vertical Ocean Temperature Profiles in Mixed Layer (Control Experiment)"
)

for i in range(len(nemo_t_grids)):
    ax = axs[i]
    ax_title = titles[i]
    nemo_t_grid = nemo_t_grids[i]
    im = ax.contourf(
        nemo_t_grid.votemper[:, :10],
        levels=12,
        vmin=9,
        vmax=12,
        discrete=True,
        transpose=True,
    )
    ax.invert_yaxis()
    ax.format(title=ax_title, xlabel="Time", ylabel="Depth [m]")
fig.colorbar(im, loc="b", title="Temperature [°C]")
fig.savefig(plotting_output_directory / "oce_temperature_profiles.pdf")

# OIFS Prognostic Variables on Lowest Model Level

In [ ]:
oifs_progvars = [*oifs_progvars_naive, oifs_progvar_cvg_swz]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atmosphere-first", "ocean-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, axs = pplt.subplots(nrows=4, spany=False, height="45em", width="75em")

ax = axs[0]
for i in range(len(colors)):
    oifs_progvar = oifs_progvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_progvar.u.sel(nlev=60),
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(
    ylabel="Zonal Wind \n$[m\; s^{{-1}}]$",
    title="",
    xlabel="Time",
    ylim=[-10, 10],
)

ax = axs[1]
for i in range(len(colors)):
    oifs_progvar = oifs_progvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_progvar.v.sel(nlev=60),
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(
    ylabel=f"Meridional Wind \n$[m\; s^{{-1}}]$",
    title="",
    xlabel="Time",
    ylim=[-10, 10],
)

ax = axs[2]
for i in range(len(colors)):
    oifs_progvar = oifs_progvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_progvar.t.sel(nlev=60) - 273.15,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(ylabel="Temperature \n$[°C]$", title="", xlabel="Time", ylim=[8, 12])

hs = []
ax = axs[3]
for i in range(len(colors)):
    oifs_progvar = oifs_progvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    h = ax.plot(
        oifs_progvar.q.sel(nlev=60) * 1e3,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
    hs.append(h)
ax.format(
    ylabel="Humidity $[g\; kg^{{-1}}]$",
    title="",
    xlabel="Time",
    ylim=[5, 10],
    xrotation=30,
)

# axs.format(suptitle="OpenIFS Prognostic Variables on Lowest Model Level")

fig.legend(hs, ncols=4, frame=False, loc="b")
fig.savefig(plotting_output_directory / "ce_progvars_oifs.pdf")

# Temperature: Study Tendencies

In [ ]:
oifs_diagvars = [*oifs_diagvars_naive, oifs_diagvar_cvg_swz]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, axs = pplt.subplots(nrows=3, spany=False, height="50em", width="70em")
axs.format(suptitle="Temperature Tendency Split in Control Experiment")

ax = axs[0]
for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.tend_temp.sel(nlev=60),
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(
    ylabel=r"$\partial_t T$ $[K\; s^{-1}]$",
    title="",
    xlabel="Time",
    ylim=[-25e-5, 25e-5],
)

ax = axs[1]
hs = []
for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    h = ax.plot(
        oifs_diagvar.tend_temp_d.sel(nlev=60),
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
    hs.append(h)
ax.format(
    ylabel=r"$(\partial_t T)_\mathrm{dyn}$ $[K\; s^{-1}]$",
    title="",
    xlabel="Time",
    ylim=[-25e-5, 25e-5],
)

ax = axs[2]
for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.tend_temp_p.sel(nlev=60),
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(
    ylabel=r"$(\partial_t T)_\mathrm{phy}$ $[K\; s^{-1}]$",
    title="",
    xlabel="Time",
    ylim=[-25e-5, 25e-5],
)

axs.format(yformatter="sci")
fig.legend(hs, frame=False, ncols=4, loc="b")
fig.savefig(plotting_output_directory / "T_tendency_split.pdf")

# Humidity Tendencies

In [ ]:
oifs_diagvars = [*oifs_diagvars_naive, oifs_diagvar_cvg_swz]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, axs = pplt.subplots(nrows=3, spany=False, height="50em", width="70em")
axs.format(suptitle="Humidity Tendency Split in Control Experiment")

ax = axs[0]
for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.tend_wat_vap.sel(nlev=60),
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(ylabel=r"$\partial_t q$ $[s^{-1}]$", title="", xlabel="Time")

ax = axs[1]
hs = []
for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    h = ax.plot(
        oifs_diagvar.tend_wat_vap_d.sel(nlev=60),
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
    hs.append(h)
ax.format(ylabel=r"$(\partial_t q)_\mathrm{dyn}$ $[s^{-1}]$", title="", xlabel="Time")

ax = axs[2]
for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.tend_wat_vap_p.sel(nlev=60),
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(ylabel=r"$(\partial_t q)_\mathrm{phy}$ $[s^{-1}]$", title="", xlabel="Time")

axs.format(yformatter="sci")
fig.legend(hs, frame=False, ncols=4, loc="b")
fig.savefig(plotting_output_directory / "q_tendency_split.pdf")

# Vertical Temperature Profile at Two Points in Time

In [ ]:
oifs_progvars = [*oifs_progvars_naive, oifs_progvar_cvg_swz]

labels = ["parallel", "atmosphere-first", "ocean-first", "converged SWR"]
colors = ["m", "c", "y", "k"]
linestyles = ["--", ":", "-.", "-"]

fig, axs = pplt.subplots(nrows=1, ncols=4, width="70em", height="30em")
# axs.format(suptitle="Vertical Atmospheric Temperature Profiles in Control Experiment")
axs.format(abc="a)")
timestamps = [
    pd.Timestamp("2014-07-02 12:00"),
    pd.Timestamp("2014-07-03 00:00"),
    pd.Timestamp("2014-07-03 12:00"),
    pd.Timestamp("2014-07-04 12:00"),
]

ax = axs[0]
ims = []
for i in range(len(oifs_progvars)):
    label = labels[i]
    color = colors[i]
    linestyle = linestyles[i]
    oifs_progvar = oifs_progvars[i]
    oifs_progvar = oifs_progvar.assign_coords(
        air_pressure=("nlev", oifs_progvar.pressure_f[0].data / 100)
    )
    oifs_progvar = oifs_progvar.swap_dims({"nlev": "air_pressure"})
    timestamp = timestamps[0]
    t_for_plotting = oifs_progvar.t.sel(time=timestamp)[45:] - 273.15
    im = ax.plot(
        t_for_plotting,
        t_for_plotting.air_pressure,
        label=label,
        color=color,
        ls=linestyle,
    )
    ims.append(im)
    ax.format(title=timestamp)

ax = axs[1]
for i in range(len(oifs_progvars)):
    label = labels[i]
    color = colors[i]
    linestyle = linestyles[i]
    oifs_progvar = oifs_progvars[i]
    oifs_progvar = oifs_progvar.assign_coords(
        air_pressure=("nlev", oifs_progvar.pressure_f[0].data / 100)
    )
    oifs_progvar = oifs_progvar.swap_dims({"nlev": "air_pressure"})
    timestamp = timestamps[1]
    t_for_plotting = oifs_progvar.t.sel(time=timestamp)[45:] - 273.15
    ax.plot(
        t_for_plotting,
        t_for_plotting.air_pressure,
        label=label,
        color=color,
        ls=linestyle,
    )
    ax.format(title=timestamp)

ax = axs[2]
for i in range(len(oifs_progvars)):
    label = labels[i]
    color = colors[i]
    linestyle = linestyles[i]
    oifs_progvar = oifs_progvars[i]
    oifs_progvar = oifs_progvar.assign_coords(
        air_pressure=("nlev", oifs_progvar.pressure_f[0].data / 100)
    )
    oifs_progvar = oifs_progvar.swap_dims({"nlev": "air_pressure"})
    timestamp = timestamps[2]
    t_for_plotting = oifs_progvar.t.sel(time=timestamp)[45:] - 273.15
    ax.plot(
        t_for_plotting,
        t_for_plotting.air_pressure,
        label=label,
        color=color,
        ls=linestyle,
    )
    ax.format(title=timestamp)

ax = axs[3]
for i in range(len(oifs_progvars)):
    label = labels[i]
    color = colors[i]
    linestyle = linestyles[i]
    oifs_progvar = oifs_progvars[i]
    oifs_progvar = oifs_progvar.assign_coords(
        air_pressure=("nlev", oifs_progvar.pressure_f[0].data / 100)
    )
    oifs_progvar = oifs_progvar.swap_dims({"nlev": "air_pressure"})
    timestamp = timestamps[3]
    t_for_plotting = oifs_progvar.t.sel(time=timestamp)[45:] - 273.15
    ax.plot(
        t_for_plotting,
        t_for_plotting.air_pressure,
        label=label,
        color=color,
        ls=linestyle,
    )
    ax.format(title=timestamp)


for ax in axs:
    ax.format(
        xlabel="Temperature [°C]",
        ylabel="Air Pressure [hPa]",
        yreverse=True,
        xlim=[-2, 12],
    )

fig.legend(ims, frame=False, ncols=4, loc="b")
fig.savefig(plotting_output_directory / "ce_air_temperature_stratification.pdf")

Level where difference of temperature is largest in panel c):

In [ ]:
max_diff_level = (
    np.abs(
        oifs_progvars[2].t.sel(time=timestamp) - oifs_progvars[3].t.sel(time=timestamp)
    )
    .argmax()
    .load()
    .data[()]
)
print(max_diff_level)

Which pressure value does this correspond to?

In [ ]:
pressure_at_max_diff = (
    oifs_progvars[0].pressure_f.sel(nlev=max_diff_level + 1).mean().load().data[()]
)
print(pressure_at_max_diff)

# Ocean Prognostic Variables

In [ ]:
nemo_ts = [*nemo_t_grids_naive, nemo_t_grid_cvg_swz]
nemo_us = [*nemo_u_grids_naive, nemo_u_grid_cvg_swz]
nemo_vs = [*nemo_v_grids_naive, nemo_v_grid_cvg_swz]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atmosphere-first", "ocean-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, axs = pplt.subplots(nrows=4, spany=False, height="45em", width="75em")

ax = axs[0]
for i in range(len(colors)):
    nemo_u = nemo_us[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        nemo_u.vozocrtx.isel(depthu=0),
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(
    ylabel=f"Zonal Current \n $[m\; s^{{-1}}]$",
    title="",
    xlabel="Time",
    ylim=[-0.15, 0.15],
)

ax = axs[1]
for i in range(len(colors)):
    nemo_v = nemo_vs[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        nemo_v.vomecrty.isel(depthv=0),
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(
    ylabel=f"Meridional Current \n $[m\; s^{{-1}}]$",
    title="",
    xlabel="Time",
    ylim=[-0.15, 0.15],
)

ax = axs[2]
for i in range(len(colors)):
    nemo_t = nemo_ts[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        nemo_t.sosstsst,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(ylabel=f"Temperature \n$[°C]$", title="", xlabel="Time", ylim=[10, 13])

hs = []
ax = axs[3]
for i in range(len(colors)):
    nemo_t = nemo_ts[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    h = ax.plot(
        nemo_t.vosaline.isel(deptht=0),
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
    hs.append(h)
ax.format(ylabel="Salinity [PSU]", title="", xlabel="Time", ylim=[32.375, 32.425])

axs.format(xrotation=30)
# axs.format(suptitle="NEMO Prognostic Variables at the Surface", )

fig.legend(hs, ncols=4, frame=False, loc="b")
fig.savefig(plotting_output_directory / "ce_progvars_nemo.pdf")

In [ ]:
for oifs_progvar in oifs_progvars:
    height_nlev_60 = oifs_progvar.height_f.sel(nlev=60)
    mean = height_nlev_60.mean().load()
    print(f"Mean Height: {mean.data[()]}")
    std = height_nlev_60.std().load()
    print(f"STD: {std.data[()]}")

In [ ]:
for oifs_progvar in oifs_progvars:
    p_nlev60 = oifs_progvar.pressure_f.sel(nlev=60)
    mean = p_nlev60.mean().load()
    print(f"Mean Pressure: {mean.data[()]}")
    mad = np.max(np.abs(p_nlev60 - mean)).load()
    print(f"MAD: {mad.data[()]}")

The pressure differences correspond to the numerical precision ($\times$ 2 because of difference between values with $\pm$ spacing)

In [ ]:
np.spacing(np.float32(mean.data[()])) * 2

Pressure at level where the vertical profile plots start:

In [ ]:
oifs_progvar.pressure_f.sel(nlev=46).min().load().data[()]